In [3]:
import numpy as np
import pandas as pd
import json
import pymongo
from clean_table import get_all_possible_keys, add_id_to_individus
path = '/media/flo/Stockage/Data/MesAides/'

In [2]:
%qtconsole
%matplotlib inline

/usr/local/lib/python3.4/dist-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


## Mongo db

In [4]:
from pymongo import MongoClient
import bson

In [7]:
client = MongoClient('localhost', 27017)
db = client['MesAides']
collection = db['situations']

In [8]:
bson_file = open(path + 'situations.bson', 'rb')
b = bson.decode_all(bson_file.read())

In [ ]:
collection.insert(b)

#### import data

In [9]:
input_data = db.situations
data = pd.DataFrame(list(input_data.find()))

## Clean Data

In [50]:
data.head(1)

,__v,_id,_updated,dateDeValeur,email,individus,logement,mobilierValue,patrimoine,phoneNumber,ressourcesYearMoins2Captured,rfr,status,token
0,3,53cd46e76e5e3202000f4616,NaN,2014-07-15,NaN,"[{'interruptedRessources': [], 'role': 'demand...","{'type': 'locataire', 'adresse': {'codePostal'...",NaN,"{'revenusDuCapital': [], 'revenusLocatifs': []}",NaN,NaN,NaN,test,NaN


#### individus

In [66]:
data.individus[0]

[{'_id': ObjectId('53cd46e76e5e3202000f4617'),
  '_id_demandeur': ObjectId('53cd46e76e5e3202000f4617'),
  'dateDeNaissance': datetime.datetime(1954, 9, 14, 0, 0),
  'enceinte': False,
  'interruptedRessources': [],
  'ressources': [{'_id': ObjectId('54465def99337c9d5413681f'),
    'montant': 720,
    'periode': '2014-04',
    'type': 'revenusSalarie'},
   {'_id': ObjectId('54465def99337c9d5413681e'),
    'montant': 720,
    'periode': '2014-05',
    'type': 'revenusSalarie'},
   {'_id': ObjectId('54465def99337c9d5413681d'),
    'montant': 720,
    'periode': '2014-06',
    'type': 'revenusSalarie'}],
  'role': 'demandeur',
  'specificSituations': []}]

In [11]:
add_id_to_individus(data.individus)

In [67]:
t_individus = pd.DataFrame(data.individus.sum())

In [70]:
t_individus.columns

Index(['_id', '_id_demandeur', 'aCharge', 'assPreconditionRemplie',
       'autoEntrepreneurActiviteType', 'autresRevenusTns',
       'autresRevenusTnsActiviteType', 'autresRevenusTnsEmployes', 'boursier',
       'boursierEnseignementSup', 'caAutoEntrepreneur', 'caAutresRevenusTns',
       'caMicroEntreprise', 'civilite', 'dateArriveeFoyer', 'dateDeNaissance',
       'dateSituationFamiliale', 'demandeurEmploi', 'departementNaissance',
       'enceinte', 'etudiant', 'firstName', 'gardeAlternee', 'inscritCaf',
       'interruptedRessources', 'isolementRecent', 'lastName', 'lienParente',
       'microEntrepriseActiviteType', 'nationalite', 'nir',
       'niveauPerteAutonomie', 'nomUsage', 'numeroAllocataire',
       'paysNaissance', 'perteAutonomie', 'place', 'ressources',
       'ressourcesApproxAnnuel', 'ressourcesYearlyApproximation', 'retraite',
       'revenusAgricolesTns', 'role', 'scolarite', 'situation',
       'situationsPro', 'specificSituations', 'statusMarital', 'statutMarital

In [92]:
t_individus.head(2)

,_id,_id_demandeur,aCharge,assPreconditionRemplie,autoEntrepreneurActiviteType,autresRevenusTns,autresRevenusTnsActiviteType,autresRevenusTnsEmployes,boursier,boursierEnseignementSup,...,situation,situationsPro,specificSituations,statusMarital,statutMarital,tauxIncapacite,tauxInvalidite,tnsActiviteType,tnsStructureType,villeNaissance
0,53cd46e76e5e3202000f4617,53cd46e76e5e3202000f4617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,53cd48556e5e3202000f4625,53cd48556e5e3202000f4625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
# -- Exemple de l'utilité de l'ajour du _id_demandeur
t_individus.groupby('_id_demandeur')['role'].value_counts().head(10)

_id_demandeur             role     
53cd46e76e5e3202000f4617  demandeur    1
53cd48556e5e3202000f4625  demandeur    1
53d0c86489d7bd02003d8459  conjoint     1
                          demandeur    1
53d0d01889d7bd02003d84a4  enfant       2
                          demandeur    1
53d0d35089d7bd02003d84ab  enfant       3
                          conjoint     1
                          demandeur    1
53d0d49189d7bd02003d84b6  demandeur    1
dtype: int64

#### logements

In [44]:
data.logement.head(1)

0    {'type': 'locataire', 'adresse': {'codePostal'...
Name: logement, dtype: object

In [45]:
# -- Replace NaN by {}
t_logements = data.logement.apply(lambda x: {} if pd.isnull(x) else x)

In [46]:
# -- Convert to DataFrame
t_logements = pd.DataFrame(t_logements.tolist())

In [52]:
t_logements['_id_demandeur'] = data['_id']

In [54]:
t_logements.head(2)

,adresse,adresseConjoint,charges,codePostal,colocation,conjointMemeAdresse,dateArrivee,inhabitantForThreeYearsOutOfLastFive,isChambre,locationType,loyer,membreFamilleProprietaire,participationFrais,pretConventionne,primoAccedant,type,_id_demandeur
0,{'codePostal': '123'},NaN,NaN,123,NaN,NaN,NaN,NaN,NaN,NaN,123,True,NaN,NaN,NaN,locataire,53cd46e76e5e3202000f4616
1,{'codePostal': '97400'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123,NaN,NaN,NaN,False,proprietaire,53cd48556e5e3202000f4624


#### patrimoines

In [41]:
data.patrimoine.head(1)

0    {'revenusDuCapital': [], 'revenusLocatifs': []}
Name: patrimoine, dtype: object

In [55]:
data.patrimoine.isnull().sum()

0

In [57]:
t_patrimoines = pd.DataFrame(data.patrimoine.tolist())

In [63]:
t_patrimoines['_id_demandeur'] = data['_id']

In [65]:
t_patrimoines.head(2)

,captured,epargneSansRevenus,epargneSurLivret,revenusDuCapital,revenusLocatifs,valeurLocativeImmoNonLoue,valeurLocativeTerrainNonLoue,_id_demandeur
0,NaN,NaN,NaN,[],[],NaN,NaN,53cd46e76e5e3202000f4616
1,NaN,NaN,NaN,[],[],NaN,NaN,53cd48556e5e3202000f4624


### Dump

In [ ]:
t_individus.to_csv(path + 'tables/individus.csv')
t_logements.to_csv(path + 'tables/logements.csv')
t_patrimoine.to_csv(path + 'tables/patrimoines.csv')